In [ ]:
!pip install sixdrepnet mediapipe

In [ ]:
import cv2
from sixdrepnet import SixDRepNet
import mediapipe as mp

In [ ]:
model = SixDRepNet()

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

my_drawing_specs = mp_drawing.DrawingSpec(color = (0, 255, 0), thickness = 1)
mp_face_mesh = mp.solutions.face_mesh

Downloading: "https://cloud.ovgu.de/s/Q67RnLDy6JKLRWm/download/6DRepNet_300W_LP_AFLW2000.pth" to /root/.cache/torch/hub/checkpoints/6DRepNet_300W_LP_AFLW2000.pth
100%|██████████| 150M/150M [00:08<00:00, 19.4MB/s]


In [ ]:
input_video_path = 'input.mp4'
output_video_path = 'output.mp4'

In [ ]:
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    with mp_face_mesh.FaceMesh(
            max_num_faces = 4,
            refine_landmarks = True,
            min_detection_confidence = 0.5,
            min_tracking_confidence = 0.5
        ) as face_mesh:

        # Process each frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
              break

            pitch, yaw, roll = model.predict(frame)
            results = face_mesh.process(frame)

            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = face_landmarks,
                    connections = mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec = None,
                    connection_drawing_spec = mp_drawing_styles
                    .get_default_face_mesh_tesselation_style()
                )

                mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = face_landmarks,
                    connections = mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec = None,
                    connection_drawing_spec = my_drawing_specs
                )


            model.draw_axis(frame, yaw, pitch, roll)

            out.write(frame)
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
process_video(input_video_path, output_video_path)